In [37]:
import requests
import pandas as pd
import json


In [38]:
# Get upcoming events
# API endpoint and headers
url = 'https://bet36528.p.rapidapi.com/next/events?sport=soccer&hours=24'
headers = {
    'x-rapidapi-host': 'bet36528.p.rapidapi.com',
    'x-rapidapi-key': '06c914e33dmsh7a12b8841983f37p11d292jsn34d8855a979c'
}

# Make the GET request
response = requests.get(url, headers=headers)

# Ensure we have a valid response
if response.status_code == 200:
    # Convert the response text to a Python dictionary
    data = json.loads(response.text)
    
    # List to hold event data
    events_data = []
    
    # Loop through the data and extract relevant details
    for tournament in data:
        for season in tournament['seasons']:
            for event in season['events'].values():
                event_info = {
                    'categoryName': tournament['categoryName'],
                    'tournamentName': tournament['name'],
                    'eventId': event['eventId'],
                    'participant1': event['participant1'],
                    'participant2': event['participant2'],
                    'date': event['date'],
                    'time': event['time'],
                    'eventStatus': event['eventStatus']
                }
                events_data.append(event_info)
    
    # Create a DataFrame from the events_data
    df = pd.DataFrame(events_data)
    
    # Display the DataFrame
    print(df.head())
else:
    print(f"Failed to retrieve data: {response.status_code}")

# You can save the dataframe to a CSV if needed
df.to_csv('events_data.csv', index=False)


  categoryName    tournamentName            eventId          participant1  \
0      Algeria           Ligue 2  id101432154457981   Skaf Khemis Miliana   
1      Algeria           Ligue 2  id101432154457979              MC Saida   
2    Argentina  Liga Profesional  id100015546657085              Platense   
3    Argentina  Liga Profesional  id100015546657091              Banfield   
4    Argentina  Liga Profesional  id100015546657093  Instituto AC Cordoba   

             participant2        date      time eventStatus  
0          NA Hussein Dey  2024-10-27  14:00:00    pre-game  
1              Js El Biar  2024-10-27  14:00:00    pre-game  
2      Argentinos Juniors  2024-10-27  18:00:00    pre-game  
3  Racing Club Avellaneda  2024-10-27  19:15:00    pre-game  
4         Rosario Central  2024-10-27  21:00:00    pre-game  


In [57]:
df[df['categoryName']=='Belgium']['eventId']

,categoryName,tournamentName,eventId,participant1,participant2,date,time,eventStatus
270,Netherlands,Eerste Divisie,id100013150869217,MVV Maastricht,Helmond Sport,2024-10-27,15:45:00,pre-game
271,Netherlands,Eredivisie,id100003750868145,FC Twente Enschede,Heracles Almelo,2024-10-27,13:30:00,pre-game
272,Netherlands,Eredivisie,id100003750868147,AFC Ajax,Willem II Tilburg,2024-10-27,15:45:00,pre-game
273,Netherlands,Eredivisie,id100003750868143,AZ Alkmaar,Go Ahead Eagles,2024-10-27,19:00:00,pre-game


In [51]:
import requests

url = "https://bet36528.p.rapidapi.com/odds_bet365"
querystring = {
    "eventId": "id100003750868145",
    "oddsFormat": "decimal",
    "raw": "false"
}

headers = {
    "x-rapidapi-host": "bet36528.p.rapidapi.com",
    "x-rapidapi-key": "06c914e33dmsh7a12b8841983f37p11d292jsn34d8855a979c"
}

response = requests.get(url, headers=headers, params=querystring)

if response.status_code == 200:
    data = response.json()
    print(data)
else:
    print(f"Request failed with status code {response.status_code}: {response.text}")

{'betradarId': 50868145, 'bookmakers': {'bet365': {'eventPath': 'https://www.bet365.com/#/AC/B1/C1/D8/E162900913', 'name': 'bet365'}}, 'date': '2024-10-27', 'eventId': 'id100003750868145', 'eventStatus': 'pre-game', 'markets': {'101': {'handicap': '0', 'marketName': 'Full Time Result', 'marketNameShort': '1X2', 'oddsType': '3Way', 'outcomes': {'101': {'bookmakers': {'bestPrice': {'price': 1.222}, 'bet365': {'bookmakerOutcomeId': '1384094955', 'price': 1.222}}, 'outcomeName': '1'}, '102': {'bookmakers': {'bestPrice': {'price': 7.0}, 'bet365': {'bookmakerOutcomeId': '1384094956', 'price': 7.0}}, 'outcomeName': 'X'}, '103': {'bookmakers': {'bestPrice': {'price': 11.0}, 'bet365': {'bookmakerOutcomeId': '1384094957', 'price': 11.0}}, 'outcomeName': '2'}}, 'payouts': {'bestPrice': 95.048, 'bet365': 95.048}}, '104': {'handicap': '0', 'marketName': 'Both Teams To Score', 'marketNameShort': 'Both Score', 'oddsType': 'Totals', 'outcomes': {'104': {'bookmakers': {'bestPrice': {'price': 1.909}, 'b

In [68]:
import requests
import pandas as pd

# Base URL and headers for the API request
url = "https://bet36528.p.rapidapi.com/odds_bet365"
headers = {
    "x-rapidapi-host": "bet36528.p.rapidapi.com",
    "x-rapidapi-key": "06c914e33dmsh7a12b8841983f37p11d292jsn34d8855a979c"
}

# List to store all rows of data
all_rows = []

# Loop over each event ID in df_event_ids['eventId']
for event_id in df['eventId']:
    # Define query parameters
    querystring = {
        "eventId": event_id,
        "oddsFormat": "decimal",
        "raw": "false"
    }

    # Send request
    response = requests.get(url, headers=headers, params=querystring)

    if response.status_code == 200:
        data = response.json()
        
        # Extract and store data
        try:
            # Iterate over each market in the response
            for market_id, market_data in data.get('markets', {}).items():
                market_name = market_data['marketName']
                market_short_name = market_data.get('marketNameShort', '')
                odds_type = market_data.get('oddsType', '')
                handicap = market_data.get('handicap', '')

                # Iterate over outcomes within each market
                for outcome_id, outcome_data in market_data.get('outcomes', {}).items():
                    outcome_name = outcome_data['outcomeName']
                    price = outcome_data['bookmakers']['bet365']['price']
                    
                    # Append each row of relevant details
                    all_rows.append({
                        'Event ID': event_id,
                        'Event Date': data.get('date', ''),
                        'Market Name': market_name,
                        'Outcome Name': outcome_name,
                        'Price': price,
                        'Handicap': handicap,
                        'Odds Type': odds_type,
                        'Bookmaker': 'bet365'
                    })
        
        except KeyError:
            print(f"Event {event_id} does not have the expected structure or data may be missing.")
    
    else:
        print(f"Request for event {event_id} failed with status code {response.status_code}: {response.text}")

# Create a DataFrame from the accumulated rows
df_events = pd.DataFrame(all_rows)

# Display the final DataFrame
print(df_events)

Request for event id101432154457981 failed with status code 400: {"message":"This match is not available for the chosen bookmakers or is already finished"}

Request for event id101432154457979 failed with status code 400: {"message":"This match is not available for the chosen bookmakers or is already finished"}

Request for event id100015546657085 failed with status code 400: {"message":"This match is not available for the chosen bookmakers or is already finished"}

Request for event id100015546657091 failed with status code 400: {"message":"This match is not available for the chosen bookmakers or is already finished"}

Request for event id100015546657093 failed with status code 400: {"message":"This match is not available for the chosen bookmakers or is already finished"}

Request for event id100015546657087 failed with status code 400: {"message":"This match is not available for the chosen bookmakers or is already finished"}

Request for event id100015546657071 failed with status cod

KeyboardInterrupt: 

In [67]:
df_merged = pd.merge(df_events, df[['eventId', 'participant1', 'participant2']], left_on='Event ID', right_on='eventId', how='left')
df_merged

,Event ID,Event Date,Market Name,Outcome Name,Price,Handicap,Odds Type,Bookmaker,eventId,participant1,participant2
0,id100013150869217,2024-10-27,Full Time Result,1,2.500,0,3Way,bet365,id100013150869217,MVV Maastricht,Helmond Sport
1,id100013150869217,2024-10-27,Full Time Result,X,4.000,0,3Way,bet365,id100013150869217,MVV Maastricht,Helmond Sport
2,id100013150869217,2024-10-27,Full Time Result,2,2.400,0,3Way,bet365,id100013150869217,MVV Maastricht,Helmond Sport
3,id100013150869217,2024-10-27,Both Teams To Score,Yes,1.571,0,Totals,bet365,id100013150869217,MVV Maastricht,Helmond Sport
4,id100013150869217,2024-10-27,Both Teams To Score,No,2.250,0,Totals,bet365,id100013150869217,MVV Maastricht,Helmond Sport
...,...,...,...,...,...,...,...,...,...,...,...
251,id100003750868143,2024-10-27,Over Under First Half,Under,1.025,3.5,Over/Under,bet365,id100003750868143,AZ Alkmaar,Go Ahead Eagles
252,id100003750868143,2024-10-27,Both Teams To Score First Half,Yes,4.000,0,Score,bet365,id100003750868143,AZ Alkmaar,Go Ahead Eagles
253,id100003750868143,2024-10-27,Both Teams To Score First Half,No,1.222,0,Score,bet365,id100003750868143,AZ Alkmaar,Go Ahead Eagles
254,id100003750868143,2024-10-27,Both Teams To Score Second Half,Yes,3.000,0,Score,bet365,id100003750868143,AZ Alkmaar,Go Ahead Eagles


In [76]:
df_merged.to_csv('bet365APIoutput.csv')